In [1]:
import tensorflow as tf
import tensorflow.keras as keras
tf.__version__

'2.0.0-rc2'

### 卷积
卷积和互相关计算是等价的，所以使用互相关代替卷积计算

In [2]:
def corr2D(X,K):
    h,w = K.shape
    out = []
    for i in range(X.shape[0]-h+1):
        for j in range(X.shape[1] - w +1):
            out.append(tf.reduce_sum(tf.multiply(X[i:i+h,j:j+w],K)))
    
    return tf.reshape(tf.stack(out),shape=(X.shape[0]-h+1,X.shape[1] - w +1))

In [3]:
x = tf.random.normal(shape=(5,5))
x

<tf.Tensor: id=5, shape=(5, 5), dtype=float32, numpy=
array([[-0.67086244, -1.249338  , -0.8696687 , -0.8753675 ,  0.17212658],
       [-0.48372144,  0.6710676 , -1.6753087 , -1.328354  ,  1.5116928 ],
       [-0.1290347 ,  1.2543837 , -0.02874732, -0.8143559 , -0.30659676],
       [-0.42957008,  1.2849354 , -0.25394708,  0.8172271 , -0.15696916],
       [ 2.9322157 , -1.2894744 ,  1.079863  ,  0.5878853 ,  1.2958982 ]],
      dtype=float32)>

In [4]:
k = tf.random.normal(shape=(3,3))
k

<tf.Tensor: id=11, shape=(3, 3), dtype=float32, numpy=
array([[-0.7105162 ,  0.7969053 ,  2.310464  ],
       [-0.24110019,  1.441733  , -0.38343838],
       [ 0.347125  , -0.07766689,  0.19343059]], dtype=float32)>

In [5]:
y = corr2D(x,k)
y

<tf.Tensor: id=77, shape=(3, 3), dtype=float32, numpy=
array([[-0.9495561 , -3.6155348 , -1.7788702 ],
       [-1.4396839 , -4.28877   ,  2.3928957 ],
       [ 4.4052362 , -4.20275   ,  0.54257274]], dtype=float32)>

### 多通道输入
多通道输入时建立和输入通道数相同的卷积核，输出是各个通道互相关计算后的和

In [6]:
def corr2D_mutil_in(X,K):
    out = []
    for x,k in zip(X,K):
        out.append(corr2D(x,k))
    
    y = tf.stack(out)
    
    return tf.reduce_sum(y,axis=0)

In [7]:
x = tf.random.normal(shape=(3,8,8))
x

<tf.Tensor: id=83, shape=(3, 8, 8), dtype=float32, numpy=
array([[[ 0.6702183 , -0.41185933, -0.41770554,  1.2909445 ,
          0.30561313, -0.09082239, -0.01464035, -0.13954517],
        [-0.63976413,  0.8535817 , -0.26961523,  0.25454444,
         -0.9422396 ,  0.95343816, -0.09432691,  1.8358234 ],
        [ 0.4477414 , -1.3823965 ,  0.98208606,  0.42875117,
         -0.504145  , -1.7440288 ,  1.0240283 ,  0.8844639 ],
        [ 0.5516646 , -0.21855839, -0.83118683, -2.1658611 ,
         -0.93818784, -0.88073933,  0.7102285 , -0.41327748],
        [ 0.28277463, -0.8142647 , -0.5995342 , -0.49852446,
         -1.1570237 , -0.23572296,  0.18558708, -0.38288212],
        [-1.2882795 , -0.6962568 ,  0.64279324,  0.2983431 ,
          1.621172  ,  0.77016264,  0.23736979, -1.2105445 ],
        [ 0.22032578,  0.02660481,  0.67669255, -1.6855978 ,
          0.1242891 ,  0.77151567,  1.3915216 ,  0.77481794],
        [-1.9619246 ,  1.3220599 , -2.4912717 , -0.02495448,
         -0.9520941 

In [8]:
k = tf.random.normal(shape=(3,3,3))
k

<tf.Tensor: id=89, shape=(3, 3, 3), dtype=float32, numpy=
array([[[ 0.37329087,  0.13370913,  0.4715222 ],
        [-1.2173921 ,  1.2188172 ,  1.6774887 ],
        [ 1.4478543 , -0.37743238, -0.14575428]],

       [[-0.6869074 ,  0.7704096 ,  0.62822   ],
        [-1.2913586 ,  0.6326922 ,  0.36846963],
        [ 1.4176263 , -1.4780763 ,  0.17998028]],

       [[-0.47253412, -0.4288232 , -1.145364  ],
        [ 0.4810103 , -0.86370313, -0.08184679],
        [-0.71141195, -0.86853373, -0.31018364]]], dtype=float32)>

In [9]:
corr2D_mutil_in(x,k)

<tf.Tensor: id=881, shape=(6, 6), dtype=float32, numpy=
array([[ 0.37146318,  1.6084745 ,  0.96355605, -1.0064038 ,  6.8749285 ,
        -0.45495248],
       [-3.8867164 , -0.6051598 , -1.1538672 , -3.91329   ,  0.62210536,
         5.530696  ],
       [ 2.8170695 , -8.066082  , -7.30146   ,  7.3880024 ,  2.1140027 ,
        -4.678872  ],
       [-6.8061576 , -6.2954893 , -9.085506  , -2.014706  ,  3.573627  ,
         3.9604692 ],
       [-2.4378166 , -5.677442  ,  0.94711566, -5.01863   ,  0.3507282 ,
        -8.934813  ],
       [-2.836907  , -1.5936394 , -6.0352597 ,  3.9985266 ,  1.7300142 ,
        -1.8911874 ]], dtype=float32)>

### 多通道输出
多通道输出时，建立和输出通道个数相同的卷积和，如果输入通道数是Ci，输出通道数是Co，卷积核大小是Kh，Kw，那么卷积和的参数有Ci × Co × Kh × Kw

In [10]:
def mutil_in_out(X,K):
    '''Co->Ci->Kh->Kw'''
    arr_k = tf.unstack(k)
    out = []
    for i in arr_k:
        out.append(corr2D_mutil_in(X,i))
    return tf.stack(out)

In [11]:
## 输入三个通道，大小是8x8
x = tf.random.normal(shape=(3,8,8))

In [12]:
#输出5个通道,输入三个通道，大小是3x3
k = tf.random.normal(shape=(5,3,3,3))

In [13]:
mutil_in_out(x,k)

<tf.Tensor: id=4859, shape=(5, 6, 6), dtype=float32, numpy=
array([[[ -2.004843  ,  -5.893652  ,   3.174364  ,  -2.1334522 ,
          -0.27104115,   5.4537454 ],
        [ -3.276765  ,  -7.6365685 ,   3.662012  ,  -6.9354734 ,
           2.3024287 ,  -4.8843727 ],
        [  1.1619089 ,   3.1474032 ,  -2.8441525 ,  12.3779125 ,
          -5.8827972 ,  -2.770535  ],
        [  1.1584239 ,  -1.4732981 ,   2.6801019 ,   0.08984685,
           2.9919417 ,   2.6620955 ],
        [  3.6778421 ,  -3.9106898 ,  -1.2861643 ,   1.681752  ,
           6.0077286 ,  -0.65976214],
        [  1.97316   ,   0.8799169 ,  -2.5793993 ,  -2.3244622 ,
           8.072494  ,  -2.9069693 ]],

       [[  0.4698838 ,   0.07080626,  -5.6248217 ,   0.9822314 ,
           2.3946815 ,   1.0716003 ],
        [  0.7432442 ,   5.6763687 ,  10.595133  ,  -0.9425566 ,
          -7.160106  ,   3.2300835 ],
        [  0.5609729 ,   5.726545  ,   6.4722996 ,   5.25488   ,
           0.52213705,  -6.951376  ],
        [ -

### 1X1卷积核
1X1卷积核的相当于全连接，它的主要作用是用来控制卷积的参数和输出的通道数

In [14]:
def mutil_in_out_1x1(X,K):
    c_i,h,w = X.shape
    c_o = K.shape[0]
    x = tf.reshape(X,shape=(c_i,h*w))
    k = tf.reshape(K,shape=(c_o,c_i))
    out = tf.matmul(k,x)
    return tf.reshape(out,shape=(c_o,h,w))

In [15]:
x = tf.random.normal(shape=(3,8,8))
k = tf.random.normal(shape=(5,3,1,1))

In [22]:
y1 = mutil_in_out_1x1(x,k)

In [23]:
y2 = mutil_in_out(x,k)

In [24]:
y1 == y2

<tf.Tensor: id=18699, shape=(5, 8, 8), dtype=bool, numpy=
array([[[ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True]],

       [[ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True, 